In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
from tqdm import tqdm

In [3]:
train = pd.read_csv("data/train.csv")
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
train["prompt"][5][2:-2]

'Construct a rap battle, in the style of Epic Rap Battles of History, with rhyme and meter and personal attacks relevant to each character, between Pikachu and Zeus. Each participant gets 2 verses. There is no announcer and Zeus goes first.'

In [5]:
train["response_a"][5][2:-2]

"[Zeus]\\nYo, it's the king of the gods on the mic\\nI control the skies, the thunder, and the lightning strike\\nYou're just a cute little rodent, Pikachu, with a spark\\nBut don't get it twisted, I'll leave you in a blur\\n\\nYou've got electric powers, but I'm the one with the might\\nI can summon lightning bolts and make them ignite\\nYou can't even compare, you're just a mere mouse\\nI'm the ruler of the gods, I'm the one who chooses\\n\\n[Pikachu]\\nYou may be the king of the gods, but I'm the one with the flow\\nI've got rhymes that'll make you bow down, Zeus, I'm the boss\\nYou may have lightning bolts, but I've got electric charm\\nI can shock you with my power, make you feel like you're not in the game\\n\\nYou may be big and bold, but I'm agile and quick\\nI can dodge your bolts and take you down with a single trick\\nYou may be powerful, but I'm the true master of the field\\nI'll leave you in the dust, Zeus, just you wait and see."

In [6]:
train["prompt"][2].split('","')[0][2:-2]

'explain function calling. how would you call a function?'

In [7]:
train["response_a"][1].split('","')[2]

'The minimal time to get a marriage license and a marriage certificate in California can vary depending on the specific requirements and process in your county. Generally, you can obtain a marriage license and a marriage certificate in California within a few days of applying, but it is best to check with your county clerk\'s office for the specific requirements and process for your county.\\n\\nIn some counties, you can obtain a marriage license and a marriage certificate on the same day, while in other counties, you may need to wait a few days. The process typically takes about 30 minutes to complete and you will need to bring identification, such as a driver\'s license or passport, and proof of your age, such as a birth certificate or a passport.\\n\\nIt is important to note that some counties in California require a waiting period between the time you obtain the marriage license and the time of the marriage. This waiting period can range from a few days to several weeks, and it is 

In [8]:
train["response_a"][1].split('","')[0][2:]

"A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage certificate, which is a document that records the marriage and is used to prove that the marriage took place. The marriage certificate is usually issued by the same government agency that issued the marriage license, and it is typically used for legal purposes, such as to change a name on a driver's license or to prove that a couple is married when applying for government benefits."

In [9]:
output_as = []
output_bs = []

for idx, row in tqdm(train.iterrows(), total=len(train)):
    try:
        prompt = row["prompt"]
        response_a = row["response_a"]
        response_b = row["response_b"]
        if prompt.find('","') != -1:
            cnt = prompt.count('","')
            prompt = row["prompt"].split('","')
            response_a = row["response_a"].split('","')
            response_b = row["response_b"].split('","')
            output_a = ""
            output_b = ""
            for sentence_idx in range(cnt + 1):
                if sentence_idx == 0:
                    output_a += prompt[0][2:].strip() + "\n"
                    output_a += response_a[0][2:].strip() + "\n"
                    output_b += prompt[0][2:].strip() + "\n"
                    output_b += response_b[0][2:].strip() + "\n"
                elif sentence_idx == cnt:
                    output_a += prompt[sentence_idx][:-2].strip() + "\n"
                    output_a += response_a[sentence_idx][:-2].strip() + "\n\n"
                    output_b += prompt[sentence_idx][:-2].strip() + "\n"
                    output_b += response_b[sentence_idx][:-2].strip() + "\n\n"
                else:
                    output_a += prompt[sentence_idx].strip() + "\n"
                    output_a += response_a[sentence_idx].strip() + "\n"
                    output_b += prompt[sentence_idx].strip() + "\n"
                    output_b += response_b[sentence_idx].strip() + "\n"
            output_as.append(output_a)
            output_bs.append(output_b)
        else:
            output_a = row["prompt"][2:-2].strip() + "\n" + row["response_a"][2:-2].strip()
            output_b = row["prompt"][2:-2].strip() + "\n" + row["response_b"][2:-2].strip()
            output_as.append(output_a)
            output_bs.append(output_b)
    except:
        print(f"Error: {idx}")
        output_as.append(" ")
        output_bs.append(" ")

  3%|▎         | 2006/57477 [00:00<00:05, 10230.73it/s]

Error: 207
Error: 385
Error: 491
Error: 763
Error: 1763
Error: 2530


  7%|▋         | 4236/57477 [00:00<00:04, 10837.51it/s]

Error: 2648
Error: 2770
Error: 2989
Error: 3152
Error: 3774
Error: 3951
Error: 4165
Error: 4197
Error: 4348


 13%|█▎        | 7584/57477 [00:00<00:04, 11090.45it/s]

...

Error: 50374
Error: 51240
Error: 51463
Error: 51664
Error: 51836
Error: 52149


 95%|█████████▌| 54759/57477 [00:05<00:00, 11112.15it/s]

Error: 52680
Error: 53194
Error: 53281
Error: 54087
Error: 54096
Error: 54765
Error: 54931
Error: 54952


100%|██████████| 57477/57477 [00:05<00:00, 10726.13it/s]

Error: 55385
Error: 55856
Error: 55912
Error: 56737
Error: 56743


In [10]:
train["full_chat_a"] = output_as
train["full_chat_b"] = output_bs

In [11]:
train.to_csv("train.csv", index=False)

In [12]:
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,full_chat_a,full_chat_b
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,Is it morally right to try to have a certain p...,Is it morally right to try to have a certain p...
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,What is the difference between marriage licens...,What is the difference between marriage licens...
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,explain function calling. how would you call a...,explain function calling. how would you call a...
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,How can I create a test set for a very rare ca...,How can I create a test set for a very rare ca...
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,What is the best way to travel from Tel-Aviv t...,What is the best way to travel from Tel-Aviv t...


In [13]:
train.drop(columns=["model_a", "model_b", "prompt", "response_a", "response_b"], axis=1, inplace=True)
train.head()

,id,winner_model_a,winner_model_b,winner_tie,full_chat_a,full_chat_b
0,30192,1,0,0,Is it morally right to try to have a certain p...,Is it morally right to try to have a certain p...
1,53567,0,1,0,What is the difference between marriage licens...,What is the difference between marriage licens...
2,65089,0,0,1,explain function calling. how would you call a...,explain function calling. how would you call a...
3,96401,1,0,0,How can I create a test set for a very rare ca...,How can I create a test set for a very rare ca...
4,198779,0,1,0,What is the best way to travel from Tel-Aviv t...,What is the best way to travel from Tel-Aviv t...


In [14]:
train["label"] = train.apply(lambda row: 0 if row["winner_model_a"] == 1 else (1 if row["winner_model_b"] == 1 else 2), axis=1)
train.head()

,id,winner_model_a,winner_model_b,winner_tie,full_chat_a,full_chat_b,label
0,30192,1,0,0,Is it morally right to try to have a certain p...,Is it morally right to try to have a certain p...,0
1,53567,0,1,0,What is the difference between marriage licens...,What is the difference between marriage licens...,1
2,65089,0,0,1,explain function calling. how would you call a...,explain function calling. how would you call a...,2
3,96401,1,0,0,How can I create a test set for a very rare ca...,How can I create a test set for a very rare ca...,0
4,198779,0,1,0,What is the best way to travel from Tel-Aviv t...,What is the best way to travel from Tel-Aviv t...,1


In [15]:
train.drop(columns=["winner_model_a", "winner_model_b", "winner_tie"], axis=1, inplace=True)
train.head()

,id,full_chat_a,full_chat_b,label
0,30192,Is it morally right to try to have a certain p...,Is it morally right to try to have a certain p...,0
1,53567,What is the difference between marriage licens...,What is the difference between marriage licens...,1
2,65089,explain function calling. how would you call a...,explain function calling. how would you call a...,2
3,96401,How can I create a test set for a very rare ca...,How can I create a test set for a very rare ca...,0
4,198779,What is the best way to travel from Tel-Aviv t...,What is the best way to travel from Tel-Aviv t...,1


In [16]:
train.to_csv("data/simplest train version.csv", index=False)

In [17]:
train[:1000].to_csv("data/sample train.csv", index=False)